In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, PageElement


# Function to get the BeautifulSoup object from a URL
def get_soup(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    else:
        print(f"Failed to fetch {url}")
        return None


# Path to the CSV file containing URLs
csv_file_path = 'urls.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

df.head()

# Extract the URLs as a list
url_list = df['URL'].tolist()  # Assuming 'URL' is the column containing the URLs

data = []

# Loop through the list of URLs and process each one
for url in url_list:
    print(url)
    soup = get_soup(url)
    if soup:

        # Initialize a dictionary to store the extracted data for this URL
        ad_data = {}

        title_el = soup.find('h1', class_='title-4206718449')
        if title_el is None:
            continue

        # Extract title, price, address, and posting date
        ad_data['Title'] = soup.find('h1', class_='title-4206718449').text.strip()

        # Extract Price
        price_elem = soup.find(class_='priceWrapper-3915768379')
        ad_data['Price'] = price_elem.text.strip() if price_elem else None

        # Extract Address
        address_elem = soup.find(itemprop='address')
        ad_data['Address'] = address_elem.text.strip() if address_elem else None

        # Extract Posting Date
        posting_date_elem = soup.find(class_='datePosted-1776470403')

        # Extract posting date if posting_date_elem is present
        if posting_date_elem:
            ad_data['Posting Date'] = posting_date_elem.time['title']
        else:
            ad_data['Posting Date'] = None

        # Extract bedrooms, bathrooms, and building type if title_attributes is present
        title_attributes = soup.find(class_='titleAttributes-183069789')
        if title_attributes:
            title_attributes = title_attributes.find_all('li', class_='noLabelAttribute-262950866')
            for attr in title_attributes:
                if 'Bedrooms' in attr.span.text:
                    ad_data['Bedrooms'] = attr.span.text.split(': ')[-1]
                elif 'Bathrooms' in attr.span.text:
                    ad_data['Bathrooms'] = attr.span.text.split(': ')[-1]
                else:
                    ad_data['Building Type'] = attr.span.text

        # Handle case where title_attributes is not present
        else:
            ad_data['Bedrooms'] = None
            ad_data['Bathrooms'] = None
            ad_data['Building Type'] = None

        # Extract attributes from Overview section
        overview_section = soup.find('h3', string='Overview')
        if overview_section:
            for li in overview_section.find_next('ul').find_all('li'):
                attribute_title_elem = li.find('h4', class_='attributeGroupTitle-889029213')
                if attribute_title_elem:
                    attribute_title = attribute_title_elem.text.strip()
                    if attribute_title == 'Utilities Included':
                        utilities_included = ', '.join(item.text.strip() for item in li.find('ul').find_all('li'))
                        ad_data['Utilities Included'] = utilities_included
                    elif attribute_title == 'Wi-Fi and More':
                        wifi_included = li.find('ul').text.strip()
                        ad_data['Wi-Fi and More'] = wifi_included
                    elif 'Parking Included' in attribute_title:
                        parking_included = li.find('dd').text.strip()
                        ad_data['Parking Included'] = parking_included
                    elif 'Agreement Type' in attribute_title:
                        agreement_type = li.find('dd').text.strip()
                        ad_data['Agreement Type'] = agreement_type
                    elif 'Pet Friendly' in attribute_title:
                        pet_friendly = li.find('dd').text.strip()
                        ad_data['Pet Friendly'] = pet_friendly

        # Assuming soup is a BeautifulSoup object
        unit_section = soup.find('h3', string='The Unit')
        if unit_section:
            for li in unit_section.find_next('ul').find_all('li'):
                if isinstance(li, PageElement):
                    try:  # Check if li is an instance of PageElement
                        attribute_title_elem = li.find('dt', class_='twoLinesLabel-2332083105')
                        if attribute_title_elem:  # Check if attribute_title_elem is not None
                            attribute_title = attribute_title_elem.text.strip()
                            print('Attribute Title:', attribute_title)
                        else:
                            print('Attribute Title not found.')
                    except Exception as e:
                        print('An error occurred:', e)

        # Extract attributes from The Building section
        building_section = soup.find('h3', string='The Building')
        if building_section:
            for li in building_section.find_next('ul').find_all('li'):
                attribute_title_elem = li.find('dt', class_='twoLinesLabel-2332083105')
                if attribute_title_elem:  # Check if attribute_title_elem is not None
                    attribute_title = attribute_title_elem.text.strip()
                    attribute_value_elem = li.find('dd', class_='twoLinesValue-2653438411')
                    attribute_value = attribute_value_elem.text.strip() if attribute_value_elem else None
                    ad_data[attribute_title] = attribute_value
                else:
                    print('Attribute title not found.')

        # Extract description
        description_elm = soup.select_one('.descriptionContainer-2067035870 p')
        ad_data['Description'] = description_elm.text.strip() if description_elm else None

        # Extract visit counter
        visit_counter_elem = soup.select_one('.visitCounter-204515568 span')
        ad_data['Visit Counter'] = visit_counter_elem.text.strip() if visit_counter_elem else None

        # Extract user info
        user_info_elem = soup.select_one('.container-3136975908 .header-1351916284')
        if user_info_elem:
            ad_data['User Info'] = user_info_elem.text.strip()
        else:
            # Extracting user info from alternative class
            alt_user_info_elem = soup.select_one('.root-3161363123 .header-1351916284')
            ad_data['User Info'] = alt_user_info_elem.text.strip() if alt_user_info_elem else None

        ad_data['url'] = url

        # Append the extracted data to the list
        data.append(ad_data)

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv('output.csv', index=False)

    # Display the DataFrame
    print(df)

https://www.kijiji.ca/v-apartments-condos/city-of-toronto/2-den-tridel-new-condo-at-bloor-and-islington/1687311830
Attribute Title: Size (sqft)
Attribute Title: Furnished
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title: Air Conditioning
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title: Smoking Permitted
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
                                           Title                        Price  \
0  2+Den Tridel New Condo at Bloor and Islington  $3,150No Utilities Included   

                           Address           Posting Date Building Type  \
0  5 Mabelle Avenue, Etobicoke, ON  March 5, 2024 2:26 AM         Condo   

  Bedrooms Bat

                                               Title  \
0      2+Den Tridel New Condo at Bloor and Islington   
1  Central Park Place - 2 Bedroom Apartment for Rent   
2  1 Bedroom  for Rent in Midtown Toronto's Leasi...   
3                                    YORK UNIVERSITY   
4  2 Bedroom for Rent in Midtown Toronto's Leasid...   
5  Bright Fully Furnished Rooms North York Yonge ...   

                           Price                                      Address  \
0    $3,150No Utilities Included              5 Mabelle Avenue, Etobicoke, ON   
1  $2,800Some Utilities Included  1294 Islington Avenue, Toronto, ON, M9A 3K2   
2  $2,300Some Utilities Included       6 Airdrie Road, East York, ON, M4G 1L7   
3                           None           22 Sheffer Terrace, North York, ON   
4  $2,875Some Utilities Included       6 Airdrie Road, East York, ON, M4G 1L7   
5                           None                       North York, ON M2M 1P1   

                 Posting Date Building 

Attribute Title: Size (sqft)
Attribute Title: Furnished
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title: Air Conditioning
Attribute Title not found.
Attribute Title: Smoking Permitted
                                               Title  \
0      2+Den Tridel New Condo at Bloor and Islington   
1  Central Park Place - 2 Bedroom Apartment for Rent   
2  1 Bedroom  for Rent in Midtown Toronto's Leasi...   
3                                    YORK UNIVERSITY   
4  2 Bedroom for Rent in Midtown Toronto's Leasid...   
5  Bright Fully Furnished Rooms North York Yonge ...   
6                              One bedroom available   
7  4 bedroom and 2 bathroom unit is for rent!  im...   
8  Renovated large beautiful apartment - Pharmacy...   

                           Price                                      Address  \
0    $3,150No Utilities Included              5 Mabelle Avenue, Etobico

                                                Title  \
0       2+Den Tridel New Condo at Bloor and Islington   
1   Central Park Place - 2 Bedroom Apartment for Rent   
2   1 Bedroom  for Rent in Midtown Toronto's Leasi...   
3                                     YORK UNIVERSITY   
4   2 Bedroom for Rent in Midtown Toronto's Leasid...   
5   Bright Fully Furnished Rooms North York Yonge ...   
6                               One bedroom available   
7   4 bedroom and 2 bathroom unit is for rent!  im...   
8   Renovated large beautiful apartment - Pharmacy...   
9   Looking for Responsible quiet female  1 Room A...   
10                            short term house rental   

                            Price  \
0     $3,150No Utilities Included   
1   $2,800Some Utilities Included   
2   $2,300Some Utilities Included   
3                            None   
4   $2,875Some Utilities Included   
5                            None   
6                            None   
7     $2,900No Util

Attribute Title: Size (sqft)
Attribute Title: Furnished
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title: Air Conditioning
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title: Smoking Permitted
Attribute title not found.
                                                Title  \
0       2+Den Tridel New Condo at Bloor and Islington   
1   Central Park Place - 2 Bedroom Apartment for Rent   
2   1 Bedroom  for Rent in Midtown Toronto's Leasi...   
3                                     YORK UNIVERSITY   
4   2 Bedroom for Rent in Midtown Toronto's Leasid...   
5   Bright Fully Furnished Rooms North York Yonge ...   
6                               One bedroom available   
7   4 bedroom and 2 bathroom unit is for rent!  im...   
8   Renovated large beautiful apartment - Pharmacy...   
9   Looking for Responsible quiet female  1 Room A...   
10   

...

                                                 Title  \
0        2+Den Tridel New Condo at Bloor and Islington   
1    Central Park Place - 2 Bedroom Apartment for Rent   
2    1 Bedroom  for Rent in Midtown Toronto's Leasi...   
3                                      YORK UNIVERSITY   
4    2 Bedroom for Rent in Midtown Toronto's Leasid...   
..                                                 ...   
546  Room sharing with one Indian girl, preferred f...   
547          Space in a shared room for rent for a boy   
548  Junior Three Bedrooms Apartment for Rent - 555...   
549             Room For Rent in Basement in Etobicoke   
550    Basement furnished room for rent in Scarborough   

                             Price  \
0      $3,150No Utilities Included   
1    $2,800Some Utilities Included   
2    $2,300Some Utilities Included   
3                             None   
4    $2,875Some Utilities Included   
..                             ...   
546                           None   

Attribute Title: Size (sqft)
Attribute Title: Furnished
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title: Air Conditioning
Attribute Title not found.
Attribute Title not found.
Attribute Title not found.
Attribute Title: Smoking Permitted
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
Attribute title not found.
                                                 Title  \
0        2+Den Tridel New Condo at Bloor and Islington   
1    Central Park Place - 2 Bedroom Apartment for Rent   
2    1 Bedroom  for Rent in Midtown Toronto's Leasi...   
3                                      YORK UNIVERSITY   
4    2 Bedroom for Rent in Midtown Toronto's Leasid...   
..                                                 ...   
548  Junior Three Bedrooms Apartment 

                                                 Title  \
0        2+Den Tridel New Condo at Bloor and Islington   
1    Central Park Place - 2 Bedroom Apartment for Rent   
2    1 Bedroom  for Rent in Midtown Toronto's Leasi...   
3                                      YORK UNIVERSITY   
4    2 Bedroom for Rent in Midtown Toronto's Leasid...   
..                                                 ...   
550    Basement furnished room for rent in Scarborough   
551  One Bedroom Apartment near Bloor West Village ...   
552  2 Bed 2 bath Condo in Downtown Toronto Availab...   
553             Renovated Jr. 1 @ Yonge and Davisville   
554                                      Room for rent   

                             Price  \
0      $3,150No Utilities Included   
1    $2,800Some Utilities Included   
2    $2,300Some Utilities Included   
3                             None   
4    $2,875Some Utilities Included   
..                             ...   
550                           None   

                                                 Title  \
0        2+Den Tridel New Condo at Bloor and Islington   
1    Central Park Place - 2 Bedroom Apartment for Rent   
2    1 Bedroom  for Rent in Midtown Toronto's Leasi...   
3                                      YORK UNIVERSITY   
4    2 Bedroom for Rent in Midtown Toronto's Leasid...   
..                                                 ...   
552  2 Bed 2 bath Condo in Downtown Toronto Availab...   
553             Renovated Jr. 1 @ Yonge and Davisville   
554                                      Room for rent   
555  LARGE 3 BEDROOM BASEMENT APARTMENT - KEELE ST ...   
556  Fantastic Room for Rent near Don Mills Subway ...   

                             Price  \
0      $3,150No Utilities Included   
1    $2,800Some Utilities Included   
2    $2,300Some Utilities Included   
3                             None   
4    $2,875Some Utilities Included   
..                             ...   
552  $3,350Some Utilities Included   

TypeError: 'NoneType' object is not subscriptable